In [1]:
#Libraries
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation
import matplotlib.pyplot as plt
import numpy as np
import requests
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from matplotlib.patches import Polygon
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import pandas as pd

from tqdm import tqdm
import requests

In [13]:
telemetry1 = pd.read_csv('decam_telemetry_20230817.csv.gz')
streaks1 = pd.read_csv('streaks_augmented_20230817.csv.gz')

In [16]:
telemetry1

,EXPNUM,BAND,TELESCOP,OBSERVAT,INSTRUME,OBS-LONG,OBS-LAT,OBS-ELEV,TELEQUIN,TIMESYS,DATE-OBS,TIME-OBS,DARKTIME,EXPTIME,RA,DEC,AZ,ZD
0,140520,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:25:35.429106,00:25:35.429106,2.10353,1.0,22:08:24.000,-32:59:17.599,116.2500,7.09
1,140521,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:29:50.667562,00:29:50.667562,2.09597,1.0,22:08:23.970,-32:55:57.799,118.9300,6.25
2,140522,u,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:46:16.576959,00:46:16.576959,31.09820,30.0,18:03:35.240,-30:02:00.600,254.2700,50.20
3,140523,u,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:47:44.370016,00:47:44.370016,31.09300,30.0,18:03:35.250,-30:02:00.802,254.1500,50.50
4,140524,u,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-10-20 00:49:47.211636,00:49:47.211636,31.09170,30.0,18:03:35.240,-30:02:00.802,253.9900,50.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506513,1228730,g,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:42:53.917964,08:42:53.917964,81.10140,80.0,03:30:36.090,-28:05:58.200,97.1620,36.05
506514,1228731,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:44:44.058111,08:44:44.058111,71.12170,70.0,03:30:36.079,-28:05:58.200,97.0003,35.66
506515,1228732,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:46:22.681357,08:46:22.681357,71.10870,70.0,03:30:36.090,-28:05:58.200,96.8515,35.30
506516,1228733,i,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-08-03 08:48:02.549261,08:48:02.549261,91.09020,90.0,03:30:36.112,-28:05:58.099,96.7036,34.94


In [15]:
streaks1

,filename,rnum,ra_1,ra_2,ra_3,ra_4,dec_1,dec_2,dec_3,dec_4,expnum,ccdnum,band
0,D00145425_g_c03_r4148p01_hstreak.fits,3,1.159330,1.160919,1.164310,1.162720,-28.724241,-28.724212,-28.857155,-28.857183,145425,3,g
1,D00145425_g_c06_r4148p01_hstreak.fits,1,0.986677,0.986711,0.644209,0.644186,-29.032018,-29.034283,-29.037208,-29.034944,145425,6,g
2,D00145425_g_c09_r4148p01_hstreak.fits,1,0.452852,0.452942,0.109842,0.109782,-29.193042,-29.199694,-29.203265,-29.196615,145425,9,g
3,D00145425_g_c15_r4148p01_hstreak.fits,1,0.286716,0.286812,0.630816,0.630741,-29.363308,-29.358405,-29.362739,-29.367644,145425,15,g
4,D00145425_g_c16_r4148p01_hstreak.fits,1,0.985852,0.987446,0.987211,0.985615,-29.216748,-29.216752,-29.366385,-29.366381,145425,16,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...
879115,D01223535_g_c05_r6224p01_hstreak.fits,1,323.500201,323.502653,323.444690,323.442238,-17.828499,-17.830897,-17.884254,-17.881855,1223535,5,g
879116,D01223535_g_c31_r6224p01_hstreak.fits,1,324.355544,324.369908,324.372363,324.357994,-18.392109,-18.391941,-18.541070,-18.541235,1223535,31,g
879117,D01223535_g_c56_r6224p01_hstreak.fits,1,322.914589,322.934330,322.849385,322.829651,-19.193809,-19.210870,-19.298406,-19.281342,1223535,56,g
879118,D01223535_g_c58_r6224p01_hstreak.fits,1,323.771850,323.783894,323.835739,323.823696,-19.214779,-19.204086,-19.255903,-19.266598,1223535,58,g


In [10]:
telemetry1.columns

Index(['EXPNUM', 'BAND', 'TELESCOP', 'OBSERVAT', 'INSTRUME', 'OBS-LONG',
       'OBS-LAT', 'OBS-ELEV', 'TELEQUIN', 'TIMESYS', 'DATE-OBS', 'TIME-OBS',
       'DARKTIME', 'EXPTIME', 'RA', 'DEC', 'AZ', 'ZD'],
      dtype='object')

In [21]:
expnum_counts = streaks1['expnum'].value_counts()
expnums_mas_8 = expnum_counts[expnum_counts > 8].index
streaks_filtradas = streaks1[streaks1['expnum'].isin(expnums_mas_8)]
telemetry1_filtrado = telemetry1[telemetry1['EXPNUM'].isin(streaks_filtradas['expnum'])]

In [22]:
streaks_filtradas

,filename,rnum,ra_1,ra_2,ra_3,ra_4,dec_1,dec_2,dec_3,dec_4,expnum,ccdnum,band
0,D00145425_g_c03_r4148p01_hstreak.fits,3,1.159330,1.160919,1.164310,1.162720,-28.724241,-28.724212,-28.857155,-28.857183,145425,3,g
1,D00145425_g_c06_r4148p01_hstreak.fits,1,0.986677,0.986711,0.644209,0.644186,-29.032018,-29.034283,-29.037208,-29.034944,145425,6,g
2,D00145425_g_c09_r4148p01_hstreak.fits,1,0.452852,0.452942,0.109842,0.109782,-29.193042,-29.199694,-29.203265,-29.196615,145425,9,g
3,D00145425_g_c15_r4148p01_hstreak.fits,1,0.286716,0.286812,0.630816,0.630741,-29.363308,-29.358405,-29.362739,-29.367644,145425,15,g
4,D00145425_g_c16_r4148p01_hstreak.fits,1,0.985852,0.987446,0.987211,0.985615,-29.216748,-29.216752,-29.366385,-29.366381,145425,16,g
...,...,...,...,...,...,...,...,...,...,...,...,...,...
879102,D01223533_g_c21_r6224p01_hstreak.fits,1,322.350759,322.367974,322.430509,322.413294,-19.778872,-19.762994,-19.822898,-19.838787,1223533,21,g
879103,D01223533_g_c27_r6224p01_hstreak.fits,1,321.914442,321.929370,322.039930,322.024991,-19.992600,-19.979549,-20.091439,-20.104500,1223533,27,g
879104,D01223533_g_c33_r6224p01_hstreak.fits,2,321.914515,321.920861,321.868432,321.862089,-20.207886,-20.214118,-20.261253,-20.255020,1223533,33,g
879105,D01223533_g_c44_r6224p01_hstreak.fits,1,323.071804,323.087515,323.208026,323.192338,-20.367516,-20.345896,-20.422967,-20.444584,1223533,44,g


In [23]:
telemetry1_filtrado

,EXPNUM,BAND,TELESCOP,OBSERVAT,INSTRUME,OBS-LONG,OBS-LAT,OBS-ELEV,TELEQUIN,TIMESYS,DATE-OBS,TIME-OBS,DARKTIME,EXPTIME,RA,DEC,AZ,ZD
1849,145434,z,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-11-02 04:53:58.963709,04:53:58.963709,31.1660,30.0,00:03:06.361,-29:47:35.696,259.1000,37.55
1856,145505,r,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-11-02 06:18:03.596634,06:18:03.596634,61.1374,60.0,02:24:27.068,-04:16:03.299,307.0800,37.89
1866,145555,i,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-11-02 07:25:16.269456,07:25:16.269456,31.0896,30.0,04:34:13.969,15:11:05.500,341.6200,47.11
1867,145558,i,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-11-02 07:32:27.251025,07:32:27.251025,91.0933,90.0,04:34:13.530,15:11:02.098,339.3600,47.63
1868,145559,i,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2012-11-02 07:34:26.345592,07:34:26.345592,91.1038,90.0,04:35:39.350,14:51:01.199,339.0600,47.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504509,1223518,g,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-07-21 04:02:44.144925,04:02:44.144925,91.1284,90.0,20:12:31.741,-01:22:56.798,28.9529,32.15
504511,1223520,g,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-07-21 04:07:18.281902,04:07:18.281902,91.0861,90.0,20:41:41.381,-07:41:56.699,45.5025,29.97
504519,1223528,g,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-07-21 04:25:07.161563,04:25:07.161563,91.1412,90.0,21:22:27.278,-20:05:40.600,73.4940,26.26
504523,1223532,g,CTIO 4.0-m telescope,CTIO,DECam,70.8149,-30.1661,2215,2000,UTC,2023-07-21 04:33:18.720174,04:33:18.720174,91.0856,90.0,21:32:30.502,-21:38:18.200,77.0527,25.96


In [17]:
expnum_counts[expnum_counts > 8] # How many times that exposure appear in the dataset

619360     61
189384     61
646812     61
1150145    61
193580     61
           ..
220686      9
915972      9
907578      9
420085      9
1069083     9
Name: expnum, Length: 38030, dtype: int64

In [20]:
expnums_mas_8 = expnum_counts[expnum_counts > 8].index.tolist() # Which are those exposures that have more than 8 streaks
#print(expnums_mas_8)

In [52]:
#expnumtry = telemetry1_filtrado['EXPNUM'].unique()  # Usaria toooooodos los datos :p
expnumtry = telemetry1_filtrado['EXPNUM'].drop_duplicates().sample(10)
                                                                   #, random_state=42) # Random exposures

In [53]:
site = EarthLocation.of_site('ctio')

In [54]:
# Contenedores de resultados
expnum_starlink   = []   # exposures que tuvieron al menos 1 STARLINK
satmap_starlink   = []   # lista de nombres Starlink para cada exposure
satmap_completo   = {}  
todos_satelites = {} 
starlinks_en_exp = []
fov_radius=1

In [55]:
for exp in tqdm(expnumtry, desc="Buscando STARLINKs"):
    fila = telemetry1_filtrado[telemetry1_filtrado['EXPNUM'] == exp].iloc[0] #row

    #print(fila)
    try:
        ra = fila['RA']
        dec = fila['DEC']
        latitude = fila ['OBS-LAT']
        longitude = fila['OBS-LONG']
        elevation = fila['OBS-ELEV']
        exptime = fila['EXPTIME']
        timeobs = fila['TIME-OBS']
        dateobs = fila['DATE-OBS']
        duration = fila['EXPTIME']
        start_time_jd = Time(fila['DATE-OBS']).jd

        # Conversión de coordenadas y fecha
        coord = SkyCoord(ra=fila['RA'], dec=fila['DEC'], unit=u.deg)
        ra_center = coord.icrs.ra.value
        dec_center = coord.icrs.dec.value
        print(f"Running query for expnum{exp}")
        
        # URL para la API
        url = f"https://dev.satchecker.cps.iau.noirlab.edu/fov/satellite-passes/?latitude={latitude}&longitude={longitude}&elevation={elevation}&start_time_jd={start_time_jd}&duration={duration}&ra={ra_center}&dec={dec_center}&fov_radius={fov_radius}&group_by=satellite"
        response = requests.get(url, timeout=60)
        response.raise_for_status()
        data = response.json()['data']['satellites']            
        
        # Guardar todos los satélites si quieres
        todos_satelites[exp] = list(data.keys())

        # Filtrar Starlink
        starlinks = [sat for sat in data if 'STARLINK' in sat.upper()]  # It was hard to find STARLINKs in a small sample, so I tried with COSMOS instead, they seem more common
        print(exp, starlinks)
        if starlinks:
            expnum_starlink.append(exp)
            starlinks_en_exp.append(starlinks)

    except Exception as e:
        print(f"⚠️ Error en expnum {exp}: {e}")
        continue

# Crear DataFrame con resultados
df_starlink = pd.DataFrame({
    'EXPNUM': expnum_starlink,
    'starlinks': starlinks_en_exp
})

# Vista rápida
print(df_starlink.head())

Buscando STARLINKs:   0%|                                                                       | 0/10 [00:00<?, ?it/s]

Running query for expnum867371


Buscando STARLINKs:  10%|██████▎                                                        | 1/10 [00:09<01:28,  9.82s/it]

867371 []
Running query for expnum907897


Buscando STARLINKs:  20%|████████████▌                                                  | 2/10 [00:21<01:27, 10.99s/it]

...

779955 []
Running query for expnum226059


Buscando STARLINKs:  90%|████████████████████████████████████████████████████████▋      | 9/10 [01:29<00:11, 11.18s/it]

226059 []
Running query for expnum627702


Buscando STARLINKs: 100%|██████████████████████████████████████████████████████████████| 10/10 [01:32<00:00,  9.28s/it]

627702 []
Empty DataFrame
Columns: [EXPNUM, starlinks]
Index: []


In [58]:
df_starlink

,EXPNUM,starlinks


In [60]:
todos_satelites

{867371: ['NOAA 16 DEB (42429)'],
 907897: ['BRITE-PL (39431)', 'EGS (AJISAI) (16908)'],
 433144: ['ARIANE 5 R/B (33057)',
  'CHINASAT 5A (CHINASTAR1) (25354)',
  'COSMOS 1275 DEB (17641)',
  'SUPERBIRD 7 (33274)',
  'WINDS (KIZUNA) (32500)'],
 431640: ['MERIDIAN 1 (29668)'],
 662421: ['DELTA 1 DEB (40764)',
  'FENGYUN 1C DEB (30112)',
  'IRIDIUM 33 DEB (33849)',
  'METEOR 2-17 DEB (28993)',
  'METEOR 2-17 DEB (38286)',
  'WASEDA-SAT3 (41933)'],
 630722: ['SL-12 DEB (18764)'],
 450490: ['CBERS 1 DEB (30783)'],
 779955: ['COSMOS 2251 DEB (34758)'],
 226059: ['COSMOS 970 DEB (10571)', 'SL-23 DEB (37756)'],
 627702: ['NAVSTAR 32 (USA 91) (22657)']}

#### Some exposures with Starlinks:

	1011967	[STARLINK-1963 (47564)] Not sure... Doesn't appear with the plot as STARLINK

	1220859	[STARLINK-3954 (52558)] Confirmed!

	1103448	[STARLINK-3738 (52117)] Not sure... Doesn't appear with the plot as STARLINK
    
#### Errors    
    
Exposure 157280 have an error with the url

Exposure 166858 have an error with the url

#### Questions

- It's possible that we are making a big restriction with the streaks restriction? (more than 8 streaks) 
- Are the STARLINK named with a different ID? Like SL?
- What's the meaning of the numbers of each satellite? STARLINK - ##### (*****)


Link of Search Satellite Database (it's correct?)

https://www.n2yo.com/satellites/?c=52

In [62]:
import requests

def get_satellite_passes(latitude, longitude, elevation, start_time_jd, duration, ra_center, dec_center, fov_radius, timeout=60, base_url="https://dev.satchecker.cps.iau.noirlab.edu"):
    """
    Fetch satellite passes using the SatChecker API.

    Parameters:
        latitude (float): Latitude in degrees.
        longitude (float): Longitude in degrees.
        elevation (float): Elevation in meters.
        start_time_jd (float): Start time in Julian Date.
        duration (float): Duration in days.
        ra_center (float): Right ascension of the field center in degrees.
        dec_center (float): Declination of the field center in degrees.
        fov_radius (float): Radius of the field of view in degrees.
        timeout (int): Timeout for the API request in seconds.
        base_url (str): Base URL for the SatChecker API.

    Returns:
        dict: Parsed JSON response from the API.

    Raises:
        requests.RequestException: If the request fails.
        ValueError: If response cannot be parsed as JSON.
    """
    endpoint = f"{base_url}/fov/satellite-passes/"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "elevation": elevation,
        "start_time_jd": start_time_jd,
        "duration": duration,
        "ra": ra_center,
        "dec": dec_center,
        "fov_radius": fov_radius,
        "group_by": "satellite"
    }

    try:
        response = requests.get(endpoint, params=params, timeout=timeout)
        response.raise_for_status()  # Raise error for HTTP errors
        print(response.url)  # Useful for debugging or logging
        return response.json()
    except requests.RequestException as e:
        print(f"Request failed: {e}")
        raise
    except ValueError:
        print("Failed to parse JSON from response.")
        raise